## Sensor.py


In [ ]:
import serial
import time

teensy_1 = serial.Serial('/dev/ttyACM0', 9600, timeout=1)
teensy_2 = serial.Serial('/dev/ttyACM1', 9600, timeout=1)

if teensy_1.is_open and teensy_2.is_open:
    print("Successfully connected to Teensy devices!")
    
sensor1_num = 0
sensor2_num = 0
sensor3_num = 0
last_time = time.time()
while True:
    if time.time()-last_time >= 0.03:
        if teensy_1.in_waiting > 0:
            data_1 = teensy_1.readline().decode('utf-8').strip()
            # print(f"Teensy 1 Data: {data_1}")
            data_1_sp = data_1.split(',')
        
            if len(data_1_sp) == 1:
                sensor3_num = data_1_sp[0].split(' ')[2]
            elif len(data_1_sp) == 2:
                sensor1_num = data_1_sp[0].split(' ')[2]
                sensor2_num = data_1_sp[1].split(' ')[2]
            else:
                print('Teensy 1 Error')
            
        if teensy_2.in_waiting > 0:
            data_2 = teensy_2.readline().decode('utf-8').strip()
            # print(f"Teensy 2 Data: {data_2}")
            data_2_sp = data_2.split(',')
            
            if len(data_2_sp) == 1:
                sensor3_num = data_2_sp[0].split(' ')[2]
            elif len(data_2_sp) == 2:
                sensor1_num = data_2_sp[0].split(' ')[2]
                sensor2_num = data_2_sp[1].split(' ')[2]
            else:
                print('Teensy 2 Error')
                
        print(f'Need sent Sensor1: {sensor1_num}, Sensor2: {sensor2_num}, Sensor3: {sensor3_num}')
        last_time = time.time()

## Snap.py

In [ ]:
import cv2
import numpy as np
import time
from PIL import ImageFont, ImageDraw, Image

def find_rect(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (7, 7), 0)
    a, thresh = cv2.threshold(blur, 220, 255, cv2.THRESH_BINARY_INV)
    cv2.imshow('Thresh', thresh)

    erode = cv2.erode(thresh, (7, 7), iterations=5)
    dilate = cv2.dilate(erode, (7, 7), iterations=5)
    contours, _ = cv2.findContours(dilate, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None
    cnt_max = max(contours, key=cv2.contourArea)
    return cnt_max

def snap_picture(cap,qrcode):
    
    ret, frame = cap.read()
    if not ret:
        print("Cannot receive frame")
        return None
    img = cv2.resize(frame,(720,420))
    ok, data, bbox, rectified = qrcode.detectAndDecodeMulti(img) 
    
    if not ok:
        input("QR code not detected. Please enter the package name:")
        
    cnt_max = find_rect(img)
    
    if cnt_max is not None:
        rect = cv2.minAreaRect(cnt_max)
        angle = rect[2]
        if angle < -45:
            angle = 90 + angle
        
        (h, w) = img.shape[:2]
        center = (w // 2, h // 2)

        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        
        box = np.int32(cv2.boxPoints(rect))
        
        pts = np.int32(cv2.transform(np.array([box]), M))[0]

        y_min = min(pts[0][0], pts[1][0], pts[2][0], pts[3][0])
        y_max = max(pts[0][0], pts[1][0], pts[2][0], pts[3][0])
        x_min = min(pts[0][1], pts[1][1], pts[2][1], pts[3][1])
        x_max = max(pts[0][1], pts[1][1], pts[2][1], pts[3][1])
        
        print('data:', data, 'x length:', x_max-x_min, ', y length:', y_max-y_min)
        cv2.drawContours(img, [box], 0, (0, 255, 0), 2)
        
    return img


cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_BRIGHTNESS, 100)
cap.set(cv2.CAP_PROP_CONTRAST, 15)
cap.set(cv2.CAP_PROP_SATURATION, 36)
cap.set(cv2.CAP_PROP_HUE, 0)
cap.set(cv2.CAP_PROP_EXPOSURE, -6)

qrcode = cv2.QRCodeDetector()  

last_time = time.time()
while True:
    if time.time() - last_time > 1:
        img = snap_picture(cap,qrcode)
        cv2.imshow('Final', img)
        last_time = time.time()
    
    k = cv2.waitKey(1)
    if k == ord('q'):
        break
    # elif k == ord('s'):
    #     img = snap_picture(cap,qrcode)
        


## IOT

In [ ]:
import socket
import time
import serial

# TCP/IP
TCP_IP = '192.168.0.152'
TCP_PORT = 5066
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

try:
    print(f"Connecting to {TCP_IP}:{TCP_PORT}")
    s.connect((TCP_IP, TCP_PORT))
except socket.error as msg:
    print(f"Failed to connect: {msg}")

print(f"Successfully connected to {TCP_IP}:{TCP_PORT}")

# Teensy
teensy_1 = serial.Serial('/dev/ttyACM0', 9600, timeout=1)
teensy_2 = serial.Serial('/dev/ttyACM1', 9600, timeout=1)

if teensy_1.is_open and teensy_2.is_open:
    print("Successfully connected to Teensy devices!")

# variable num
sensor1_num = 0
sensor2_num = 0
sensor3_num = 0
read_time = time.time()
send_time = time.time()

while True:
    if time.time()-read_time >= 0.03:
        if teensy_1.in_waiting > 0:
            data_1 = teensy_1.readline().decode('utf-8').strip()
            # print(f"Teensy 1 Data: {data_1}")
            data_1_sp = data_1.split(',')
        
            if len(data_1_sp) == 1:
                sensor3_num = data_1_sp[0].split(' ')[2]
            elif len(data_1_sp) == 2:
                sensor1_num = data_1_sp[0].split(' ')[2]
                sensor2_num = data_1_sp[1].split(' ')[2]
            else:
                print('Teensy 1 Error')
            
        if teensy_2.in_waiting > 0:
            data_2 = teensy_2.readline().decode('utf-8').strip()
            # print(f"Teensy 2 Data: {data_2}")
            data_2_sp = data_2.split(',')
            
            if len(data_2_sp) == 1:
                sensor3_num = data_2_sp[0].split(' ')[2]
            elif len(data_2_sp) == 2:
                sensor1_num = data_2_sp[0].split(' ')[2]
                sensor2_num = data_2_sp[1].split(' ')[2]
            else:
                print('Teensy 2 Error')
                
        print(f'Need sent Sensor1: {sensor1_num}, Sensor2: {sensor2_num}, Sensor3: {sensor3_num}')
        read_time = time.time()
    
    if time.time() - send_time >= 0.2:
        send_data = f'Sensor1: {sensor1_num},Sensor2: {sensor2_num},Sensor3: {sensor3_num}'
        s.send(send_data.encode('utf-8'))
        send_time = time.time()
